In [1]:
import numpy as np
import os
def dp_matching(feature_1, feature_2):
    '''
    DP Matching을 수행한다
    입력:
        feature_1: 비교할 특징값 배열 1
        feature_2: 비교한 특징값 배열 2
    출력:
        total_cost: 최단 경로의 총 비용
        min_path: 최단 경로 프레임 대응
        
    '''
    # 프레임 수와 차원 수를 추출
    (nframes_1, num_dims) = np.shape(feature_1)
    nframes_2 = np.shape(feature_2)[0]
    
    # 거리 (비용) 행렬을 계산한다
    distance = np.zeros((nframes_1, nframes_2))
    for n in range(nframes_1):
        for m in range(nframes_2):
            # feature_1의 n 번째 프레임과
            # feature_2의 m 번째 프레임의
            # 유클리드 거리 제곱을 계산
            distance[n, m] = np.sum((feature_1[n] - feature_2[m])**2)
            
    
    # 누적 비용 행렬
    cost = np.zeros((nframes_1, nframes_2))
    #이동 종류 (세로/사선/가로) 를 기록하는 행렬
    # 0: 세로 이동, 1: 사선 이동, 2: 가로이동
    track = np.zeros((nframes_1, nframes_2), np.int16)

    
    # 시작지점의 거리
    cost[0, 0] = distance[0, 0]
    
    # 0번째 행: 반드시 세로로 (아래로) 이동한다
    for n in range(1, nframes_1):
        cost[n, 0] = cost[n-1, 0] + distance[n, 0]
        track[n, 0] = 0
    
    # 0번째 열: 반드시 가로로 (우측으로) 이동한다
    for m in range(1, nframes_2):
        cost[0, m] = cost[0, m-1] + distance[0, m]
        track[0, m] = 2
        
    # 그외 : 가로 세로 사선 중에 최소 비용으로 이동한다.
    for n in range(1, nframes_1):
        for m in range(1, nframes_2):
            # 세로로 이동했을때 누적 비용
            vertical = cost[n-1, m] + distance[n, m]
            # 사선으로 이동했을 때 누적 비용
            # (사선은 2배 가중치를 부여함)
            diagonal = cost[n-1, m-1] + 2 * distance[n, m]
            # 가로로 이동했을 때 누적 비용
            horizontal = cost[n, m-1] + distance[n, m]
            
            # 누적 비용이 최소인 이동 경로를 선택한다
            candidate = [vertical, diagonal, horizontal]
            transition = np.argmin(candidate)
            
            # 누적 비용과 이동 방향을 기록한다
            cost[n, m] = candidate[transition]
            track[n, m] = transition
            
        
    #총 비용은 cost 행렬의 최종행 X 최종 열 값
    #특징값은 프레임수로 정규화
    total_cost = cost[-1, -1] / (nframes_1 + nframes_2)
    
    #Back Track
    # 끝에서 track 값을 기준으로 역추적하여
    #최소비용 경로를 구한다
    min_path= []
    n = nframes_1 - 1
    m = nframes_2 - 1
    while True:
        #현재 프레임 위치를 min_path 에 추가한다
        min_path.append([n,m])
        
        # 시작 지점에 도달하면 종료한다
        if n == 0 and m == 0:
            break
            
        # track 값을 확인한다
        if track[n, m] == 0:
            #세로 이동일 경우
            n-=1
        elif track[n, m] == 1:
            # 사선 이동일 경우
            n -=1
            m -=1
        else:
            #가로 이동일 경우
            m-=1
    #min_path 를 역순으로 교체한다
    min_path = min_path[::-1]
    
    # 총비용과 최종 경로를 출력한다
    return total_cost, min_path

if __name__ == '__main__':
    #인식 대상의 세트 번화와 발화 번호
    query_set = 1
    query_utt = 9
    
    #knearest neighbor 매개변수
    K=3
    #MFCC 차원수
    num_dims= 13
    #총 세트 수
    num_set = 5
    #발화 종류 수
    num_utt = 10
    
    # 특징값 데이터를 특징값 파일에서 읽어온다
    query_file = './mfcc/REPEAT500_set%d_%03d.bin' % (query_set, query_utt)
    query = np.fromfile(query_file, dtype=np.float32)
    query = query.reshape(-1,num_dims)
    
    cost = []
    for set_id in range(1, num_set+1):
        for utt_id in range(1, num_utt+1):
            #query와 동일 세트는 비교하지 않음
            if set_id == query_set:
                continue
            #비교 대상의 특징값을 읽어온다
            target_file = './mfcc/REPEAT500_set%d_%03d.bin' %(set_id, utt_id)
            target = np.fromfile(target_file, dtype=np.float32)
            target = target.reshape(-1, num_dims)
            
            #DP Matching 실시
            tmp_cost, tmp_path = dp_matching(query, target)
            
            cost.append({'utt':utt_id,'set':set_id,'cost':tmp_cost})
            
    #비용 순으로 정렬
    cost = sorted(cost, key=lambda x:x['cost'])
    #비용 순위를 표시
    for n in range(len(cost)):
        print('%d: utt:%d, set:%d, cost: %.3f' % (n+1, cost[n]['utt'], cost[n]['set'], cost[n]['cost']))
    
    voting = np.zeros(num_utt, np.int16)
    for n in range(K):
        voting[cost[n]['utt']-1] +=1
    
    #투표로 가장 많은 값을 얻은 발화 ID를 출력
    max_voted = np.argmax(voting) + 1
    print('Estimated utterance id = %d' % max_voted)

1: utt:9, set:3, cost: 5166.617
2: utt:9, set:5, cost: 5630.207
3: utt:9, set:2, cost: 7636.151
4: utt:9, set:4, cost: 8371.491
5: utt:1, set:4, cost: 10948.727
6: utt:4, set:2, cost: 11030.011
7: utt:8, set:4, cost: 12079.891
8: utt:8, set:3, cost: 12105.722
9: utt:1, set:3, cost: 12215.102
10: utt:4, set:4, cost: 12228.659
11: utt:1, set:5, cost: 12263.464
12: utt:4, set:5, cost: 13073.162
13: utt:2, set:3, cost: 13288.440
14: utt:8, set:5, cost: 13353.192
15: utt:2, set:5, cost: 13441.711
16: utt:2, set:4, cost: 13741.787
17: utt:3, set:5, cost: 13788.769
18: utt:10, set:3, cost: 13837.919
19: utt:7, set:5, cost: 14182.015
20: utt:1, set:2, cost: 14263.879
21: utt:3, set:4, cost: 14357.661
22: utt:7, set:2, cost: 14530.841
23: utt:10, set:5, cost: 14563.917
24: utt:2, set:2, cost: 14628.543
25: utt:8, set:2, cost: 14877.179
26: utt:7, set:4, cost: 15154.636
27: utt:6, set:2, cost: 15182.629
28: utt:7, set:3, cost: 15284.148
29: utt:6, set:5, cost: 15631.678
30: utt:10, set:4, cost: 